In [ ]:
#!accelerate launch --mixed_precision fp16 ./train_accel.py --config_name EXP_02

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V1

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V2

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V0

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V1

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V2

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPH

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPHV2

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V3

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V5

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V3

Training with config: {'__module__': 'config', 'EXP_NAME': 'EXP_12', 'LOSS_FUNC': <class 'icecube.models.LogCoshLoss'>, 'TRN_DATASET': <class 'icecube.dataset.HuggingFaceDatasetV5'>, 'VAL_DATASET': <class 'icecube.dataset.HuggingFaceDatasetV5'>, 'TRN_BATCH_RANGE': (1, 100), 'VAL_BATCH_RANGE': (622, 627), 'EPOCHS': 10, '__doc__': None}
epoch     train_loss  valid_loss  val_metric
/usr/local/lib/python3.8/dist-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(
wandb: Currently logged in as: drhb (kaggle-hi). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.9
wandb: Run data is saved locally in /opt/slh/icecube/wandb/run-20230204_013216-mc7ax3zy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run EXP_12
wandb: ⭐️ View project at ht